<a href="https://colab.research.google.com/github/FacundoR26/groundwater-dependent-forests-sanjuan/blob/main/Identificacion_bosques2026.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Identificacion de bosques asociados a la napa usando RF**

# ***Obtencion de indices NDVI y NDWI***
Obtendremos los indices para la estacion seca y humeda desde el año 2015 hasta el 2025 y los visualizaremas con la libreria geemap.

In [ ]:
import ee
import geemap
import pandas as pd

### Funcion que calcula NDVI y NDWI

In [ ]:
# Funcion para calcular NDVI y NDWI
def add_indices(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI')
    return image.addBands([ndvi, ndwi])

### Definimos el roi

In [ ]:
# Replace 'users/your_username/your_asset_path' with the actual asset ID of your shapefile in Earth Engine
shapefile_asset_id = 'users/facu_ruarte/puntos_freatica_exp'

# Load the shapefile asset from Earth Engine
try:
    roi_feature_collection = ee.FeatureCollection(shapefile_asset_id)
    print(f"Successfully loaded shapefile asset: {shapefile_asset_id}")

    # Extract the geometry of the first feature
    # Assuming the shapefile asset contains at least one feature
    roi_geometry = roi_feature_collection.first().geometry()
    print("Extracted geometry from the first feature in the collection.")


    # Check if Map object exists, if not, create one
    if 'Map' not in locals():
        Map = geemap.Map()
        print("geemap.Map() object created.")

    # Center the map on the loaded shapefile geometry
    Map.centerObject(roi_geometry)
    print("Centered map on the loaded shapefile geometry.")

    # You can optionally add the shapefile to the map as a layer for visualization
    Map.addLayer(roi_feature_collection, {}, 'Region of Interest (from Asset)')
    print("Added shapefile layer to the map.")

    # Display the map
    display(Map)

except ee.EEException as e:
    print(f"Error loading Earth Engine asset: {e}")
    print(f"Please ensure the asset ID '{shapefile_asset_id}' is correct and the asset is publicly accessible or in your Earth Engine account.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

### Funcion para obtener medianas por estacion
La estacion seca estara compuesta por los meses Febrero, Marzo y Abril, y la estacion humeda por los meses Julio, Agosto y Septiembre, usaremos la mediana para obtener un valor representativo para cada estacion.

In [ ]:
# Funcion para obtener medianas por estacion
def get_season_median(year, months, season_name):
    start = ee.Date.fromYMD(year, months[0], 1)
    end = ee.Date.fromYMD(year, months[-1], 30)

    collection = (ee.ImageCollection("COPERNICUS/S2_SR")
                  .filterDate(start, end)
                  .filterBounds(roi_geometry) # Use the extracted geometry
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)) # Further relaxed cloud filter
                  .map(add_indices)
                  .select(['NDVI', 'NDWI'])) # Select only the calculated index bands

    median = collection.median().clip(roi_geometry) # Clip using the extracted geometry
    return median.set({'year': year, 'season': season_name})

# Generar imagenes por año y estacion desde el 2015 hasta el 2025
dry_months = [2, 3, 4]
wet_months = [7, 8, 9]

dry_list = []
wet_list = []

for year in range(2019, 2026):
    dry = get_season_median(year, dry_months, 'dry')
    wet = get_season_median(year, wet_months, 'wet')
    dry_list.append(dry)
    wet_list.append(wet)

dry_collection = ee.ImageCollection(dry_list)
wet_collection = ee.ImageCollection(wet_list)

## Visualizamos el NDVI y NDWI de un año en especifico
Se usa el stack de bandas obtenido en la linea anterior

In [ ]:
# Visualizar NDVI y NDWI de año específico
year_to_show = 2022 # Especificar el año a observar

dry_collection_year = dry_collection.filter(ee.Filter.eq('year', year_to_show))
wet_collection_year = wet_collection.filter(ee.Filter.eq('year', year_to_show))

# Adjusted NDVI parameters to include negative values
ndvi_params = {'min': -1, 'max': 1, 'palette': ['red','white', 'green']}
ndwi_params = {'min': -1, 'max': 1, 'palette': ['brown','white', 'cyan']}

# Add checks to ensure collections are not empty before getting the first image and adding layers
if dry_collection_year.size().getInfo() > 0:
    dry_img = dry_collection_year.first()
    # Clip the image first using the extracted geometry, then select bands
    dry_img_clipped = dry_img.clip(roi_geometry)
    # Swapped "Seco" and "Húmedo" labels (interpreted)
    Map.addLayer(dry_img_clipped.select('NDVI'), ndvi_params, f'NDVI Húmedo (interpreted) {year_to_show}')
    Map.addLayer(dry_img_clipped.select('NDWI'), ndwi_params, f'NDWI Húmedo (interpreted) {year_to_show}')
else:
    print(f"No data available with NDVI and NDWI bands for dry season {year_to_show} in the selected region.")

if wet_collection_year.size().getInfo() > 0:
    wet_img = wet_collection_year.first()
    # Clip the image first using the extracted geometry, then select bands
    wet_img_clipped = wet_img.clip(roi_geometry)
    # Swapped "Seco" and "Húmedo" labels (interpreted)
    Map.addLayer(wet_img_clipped.select('NDVI'), ndwi_params, f'NDVI Seco (interpreted) {year_to_show}')
    Map.addLayer(wet_img_clipped.select('NDWI'), ndwi_params, f'NDWI Seco (interpreted) {year_to_show}')
else:
    print(f"No data available with NDVI and NDWI bands for wet season {year_to_show} in the selected region.")

# Observar valores minimos y maximos
print("Layers on the map:")
for layer in Map.layers:
    print(layer.name)

# Optional: Inspect image data within the drawn region (may take some time)
print(f"\nInspecting image data for {year_to_show} dry season:")
# Use roi_geometry for reducing region
if dry_collection_year.size().getInfo() > 0:
    dry_stats = dry_collection_year.first().select(['NDVI', 'NDWI']).reduceRegion(
         reducer=ee.Reducer.minMax(),
         geometry=roi_geometry,
         scale=100, # Adjust scale as needed
         maxPixels=1e9
    )
    print(dry_stats.getInfo())
else:
    print("Dry season image collection for the specified year is empty. Cannot compute statistics.")


print(f"\nInspecting image data for {year_to_show} wet season:")
# Use roi_geometry for reducing region
if wet_collection_year.size().getInfo() > 0:
    wet_stats = wet_collection_year.first().select(['NDVI', 'NDWI']).reduceRegion(
         reducer=ee.Reducer.minMax(),
         geometry=roi_geometry,
         scale=100, # Adjust scale as needed
         maxPixels=1e9
    )
    print(wet_stats.getInfo())
else:
     print("Wet season image collection for the specified year is empty. Cannot compute statistics.")

In [ ]:
# Display the map
display(Map)

# ***Unificación*** y Preparación de los Datos para el Modelo

In [ ]:
!pip install geopandas

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import geopandas as gpd

## Specify the file path


In [ ]:
geojson_file_path = '/content/drive/My Drive/earth_engine_exports/puntos_freatica_expanded.geojson'

## Read the geojson file


In [ ]:
gdf_puntos_muestreo = gpd.read_file(geojson_file_path)
display(gdf_puntos_muestreo.head())

## Feature extraction and presence/absence points adjust
Fusion de las variables en un solo dataframe.

In [ ]:
# --- 1. Feature Extraction (DEM, TWI, NDVI, NDWI) ---

if gdf_puntos_muestreo is not None and roi_geometry is not None:
    # Drop duplicate columns from gdf_puntos_muestreo before converting to EE FeatureCollection
    gdf_puntos_muestreo = gdf_puntos_muestreo.loc[:,~gdf_puntos_muestreo.columns.duplicated()]
    print("Dropped duplicate columns from gdf_puntos_muestreo.")

    # Load the SRTM image
    srtm_image_id = 'USGS/SRTMGL1_003'
    srtm_image = ee.Image(srtm_image_id)
    print("SRTM image loaded.")

    # Filter SRTM image by ROI
    filtered_srtm_image = srtm_image.clip(roi_geometry)
    print("SRTM image filtered by ROI.")

    # Compute Slope and TWI
    elevation_band = filtered_srtm_image.select('elevation')
    slope_image = ee.Terrain.slope(elevation_band)
    print("Slope calculated.")

    # Calculate TWI: ln(Flow Accumulation / tan(Slope))
    # Ensure slope is in radians and avoid division by zero
    # Corrected: Use ee.Image.tan() and ee.Image.log()
    slope_radians = slope_image.divide(180).multiply(ee.Number(math.pi))
    # Replace zero or near-zero slopes with a small positive number
    min_slope_deg = 0.001
    slope_corrected = slope_image.max(min_slope_deg)
    slope_radians_corrected = slope_corrected.divide(180).multiply(ee.Number(math.pi))
    tan_slope = slope_radians_corrected.tan()

    # Replace zero or near-zero flow accumulation values
    min_flow_acc = 1e-5 # Use a very small positive number
    # Ensure flow_accumulation_ee_image exists and has the band 'b1'
    if flow_accumulation_ee_image is not None and 'b1' in flow_accumulation_ee_image.bandNames().getInfo():
        flow_accumulation_corrected = flow_accumulation_ee_image.select('b1').max(min_flow_acc) # Select the band if needed
        # Calculate TWI
        twi_image = flow_accumulation_corrected.divide(tan_slope).log().rename('TWI')
        print("TWI calculated.")

        # Add DEM and TWI layers to map (optional visualization)
        elevation_viz = {'min': 0, 'max': 4000, 'palette': ['006633', 'E5FFCC', '662A00', 'FF8040', '000000']}
        # Dynamically calculate TWI viz min/max if possible, otherwise use defaults
        try:
            twi_min_max = twi_image.reduceRegion(reducer=ee.Reducer.minMax(), geometry=roi_geometry, scale=30, maxPixels=1e9)
            twi_min = twi_min_max.get('TWI_min').getInfo() if twi_min_max.get('TWI_min') is not None else 0
            twi_max = twi_min_max.get('TWI_max').getInfo() if twi_min_max.get('TWI_max') is not None else 10
            twi_viz = {'min': twi_min, 'max': twi_max, 'palette': ['brown', 'yellow', 'green', 'cyan', 'blue']}
            Map.addLayer(filtered_srtm_image.select('elevation'), elevation_viz, 'SRTM Elevation Model')
            Map.addLayer(twi_image, twi_viz, 'Topographic Wetness Index (TWI)')
            print("DEM and TWI layers added to map.")
        except Exception as e:
             print(f"Warning: Could not dynamically calculate TWI min/max or add layers: {e}")
             # Add layers with default visualization if dynamic calculation fails
             twi_viz_default = {'min': 0, 'max': 10, 'palette': ['brown', 'yellow', 'green', 'cyan', 'blue']}
             Map.addLayer(filtered_srtm_image.select('elevation'), elevation_viz, 'SRTM Elevation Model')
             Map.addLayer(twi_image, twi_viz_default, 'Topographic Wetness Index (TWI) (Default Viz)')
             print("DEM and TWI layers added to map with default visualization.")



        # Convert the GeoDataFrame to an Earth Engine FeatureCollection for extraction
        points_ee = geemap.gdf_to_ee(gdf_puntos_muestreo)
        print("GeoDataFrame converted to Earth Engine FeatureCollection.")

        # Extract DEM and TWI values at sample points
        # Ensure the band names exist in the images before selecting
        dem_values_ee = filtered_srtm_image.select('elevation').reduceRegions(
            collection=points_ee, reducer=ee.Reducer.first(), scale=30
        )
        twi_values_ee = twi_image.select('TWI').reduceRegions(
            collection=points_ee, reducer=ee.Reducer.first(), scale=30
        )
        dem_df = geemap.ee_to_df(dem_values_ee)
        twi_df = geemap.ee_to_df(twi_values_ee)
        print("Extracted DEM and TWI values.")
        # Drop original 'Nelfle' column if it exists and merge DEM/TWI
        if 'Nelfle' in gdf_puntos_muestreo.columns:
            gdf_puntos_muestreo = gdf_puntos_muestreo.drop(columns=['Nelfle'])
        # Ensure 'first' column exists before renaming
        if 'first' in dem_df.columns:
            dem_df = dem_df.rename(columns={'first': 'DEM'})
        if 'first' in twi_df.columns:
             twi_df = twi_df.rename(columns={'first': 'TWI'})

        # Merge based on 'Site' or other common columns
        # Check if 'Site' column exists in both dataframes before merging
        if 'Site' in gdf_puntos_muestreo.columns and 'Site' in dem_df.columns and 'Site' in twi_df.columns:
            merged_gdf = gdf_puntos_muestreo.merge(dem_df[['Site', 'DEM']], on='Site', how='left')
            merged_gdf = merged_gdf.merge(twi_df[['Site', 'TWI']], on='Site', how='left')
            gdf_puntos_muestreo = merged_gdf # Update the main GeoDataFrame
            print("DEM and TWI merged into GeoDataFrame.")
        else:
            print("Warning: 'Site' column not found in all necessary dataframes for DEM/TWI merge. Skipping merge.")
            # Handle cases where merge cannot be done
            # If merge fails, gdf_puntos_muestreo will still have original columns + possibly Nelfle

        # --- Generate multi-year median images (NDVI and NDWI) ---
        # Function to calculate NDVI and NDWI (already defined above, but included again for clarity in this single block)
        def add_indices(image):
            ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
            ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI')
            return image.addBands([ndvi, ndwi])

        # Function to get median by season (already defined above, but included again for clarity)
        def get_season_median(year, months, season_name):
            start = ee.Date.fromYMD(year, months[0], 1)
            end = ee.Date.fromYMD(year, months[-1], 30)
            collection = (ee.ImageCollection("COPERNICUS/S2_SR")
                          .filterDate(start, end)
                          .filterBounds(roi_geometry)
                          .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))
                          .map(add_indices)
                          .select(['NDVI', 'NDWI']))
            median = collection.median().clip(roi_geometry)
            return median.set({'year': year, 'season': season_name})

        # Generate multi-year median collections
        dry_months = [2, 3, 4]
        wet_months = [7, 8, 9]
        dry_list = []
        wet_list = []
        for year in range(2019, 2026):
            dry = get_season_median(year, dry_months, 'dry')
            wet = get_season_median(year, wet_months, 'wet')
            dry_list.append(dry)
            wet_list.append(wet)
        dry_collection = ee.ImageCollection(dry_list)
        wet_collection = ee.ImageCollection(wet_list)
        print("Multi-year NDVI and NDWI median collections created.")

        # Extract multi-year NDVI and NDWI values at sample points
        all_ndvi_ndwi_data = []
        for year in range(2019, 2026):
            # Filter collections by year
            dry_img_year_collection = dry_collection.filter(ee.Filter.eq('year', year))
            wet_img_year_collection = wet_collection.filter(ee.Filter.eq('year', year))

            # Check if collections are not empty for the year
            if dry_img_year_collection.size().getInfo() > 0:
                dry_img_year = dry_img_year_collection.first()
                dry_extracted = dry_img_year.select(['NDVI', 'NDWI']).reduceRegions(
                    collection=points_ee, reducer=ee.Reducer.first(), scale=10
                )
                dry_list = dry_extracted.getInfo()['features']
                for feature in dry_list:
                    properties = feature['properties']
                    properties['year'] = year
                    properties['season'] = 'dry'
                    all_ndvi_ndwi_data.append(properties)
            else:
                print(f"Warning: Dry season collection for year {year} is empty. Skipping extraction.")

            if wet_img_year_collection.size().getInfo() > 0:
                wet_img_year = wet_img_year_collection.first()
                wet_extracted = wet_img_year.select(['NDWI', 'NDVI']).reduceRegions( # Ensure correct bands are selected
                    collection=points_ee, reducer=ee.Reducer.first(), scale=10
                )
                wet_list = wet_extracted.getInfo()['features']
                for feature in wet_list:
                    properties = feature['properties']
                    properties['year'] = year
                    properties['season'] = 'wet'
                    all_ndvi_ndwi_data.append(properties)
            else:
                 print(f"Warning: Wet season collection for year {year} is empty. Skipping extraction.")


        ndvi_ndwi_df = pd.DataFrame(all_ndvi_ndwi_data)
        print("Multi-year NDVI and NDWI values extracted.")

        # Combine all extracted data
        # Pivot ndvi_ndwi_df
        # Ensure ndvi_ndwi_df is not empty and has expected columns
        if not ndvi_ndwi_df.empty and 'Site' in ndvi_ndwi_df.columns and 'year' in ndvi_ndwi_df.columns and 'season' in ndvi_ndwi_df.columns and 'NDVI' in ndvi_ndwi_df.columns and 'NDWI' in ndvi_ndwi_df.columns:
            ndvi_ndwi_pivot_df = ndvi_ndwi_df[['Site', 'year', 'season', 'NDVI', 'NDWI']].copy()
            ndvi_ndwi_pivot_df['year_season'] = ndvi_ndwi_pivot_df['year'].astype(str) + '_' + ndvi_ndwi_pivot_df['season']
            ndvi_ndwi_pivot_df = ndvi_ndwi_pivot_df.drop(columns=['year', 'season'])
            # Check for duplicate index entries before setting index and unstacking
            if not ndvi_ndwi_pivot_df.set_index(['Site', 'year_season']).index.is_unique:
                 print("Warning: Duplicate 'Site', 'year_season' combinations found. Aggregating before pivoting.")
                 # Aggregate by taking the mean or first value for duplicates if necessary
                 ndvi_ndwi_pivot_df = ndvi_ndwi_pivot_df.groupby(['Site', 'year_season']).mean().reset_index() # Example: take mean

            ndvi_ndwi_pivot_df = ndvi_ndwi_pivot_df.set_index(['Site', 'year_season'])
            pivoted_ndvi_ndwi = ndvi_ndwi_pivot_df.unstack()
            pivoted_ndvi_ndwi.columns = ['_'.join(col).strip() for col in pivoted_ndvi_ndwi.columns.values]
            pivoted_ndvi_ndwi = pivoted_ndvi_ndwi.reset_index()
            # Merge with GeoDataFrame (which now has DEM and TWI)
            # Ensure gdf_puntos_muestreo is not None before merging
            if gdf_puntos_muestreo is not None and 'Site' in pivoted_ndvi_ndwi.columns and 'Site' in gdf_puntos_muestreo.columns:
                 data_for_modeling = gdf_puntos_muestreo.merge(pivoted_ndvi_ndwi, on='Site', how='left')
                 # Drop duplicate DEM and TWI columns if they exist and rename the kept ones
                 # Assuming DEM_x and TWI_x are the desired columns after merge
                 if 'DEM_x' in data_for_modeling.columns and 'DEM_y' in data_for_modeling.columns:
                     data_for_modeling = data_for_modeling.drop(columns=['DEM_y'])
                     data_for_modeling = data_for_modeling.rename(columns={'DEM_x': 'DEM'})
                 elif 'DEM_y' in data_for_modeling.columns: # Handle case if only DEM_y exists after some merge issue
                      data_for_modeling = data_for_modeling.rename(columns={'DEM_y': 'DEM'})

                 if 'TWI_x' in data_for_modeling.columns and 'TWI_y' in data_for_modeling.columns:
                     data_for_modeling = data_for_modeling.drop(columns=['TWI_y'])
                     data_for_modeling = data_for_modeling.rename(columns={'TWI_x': 'TWI'})
                 elif 'TWI_y' in data_for_modeling.columns: # Handle case if only TWI_y exists after some merge issue
                      data_for_modeling = data_for_modeling.rename(columns={'TWI_y': 'TWI'})


                 print("All data combined for modeling.")
                 display(data_for_modeling.head())
            else:
                 print("Warning: Cannot combine data for modeling. Check if gdf_puntos_muestreo or pivoted_ndvi_ndwi is valid or if 'Site' column exists.")
                 data_for_modeling = None # Set to None if merge fails
        else:
            print("Warning: Cannot pivot NDVI/NDWI data. Check if ndvi_ndwi_df is empty or missing columns.")
            data_for_modeling = gdf_puntos_muestreo # Use only DEM/TWI data if NDVI/NDWI extraction/pivoting fails

    else:
        print("Warning: TWI image could not be calculated. Skipping TWI extraction and combining.")
        # Proceed with data_for_modeling using only DEM and original columns if TWI failed
        data_for_modeling = gdf_puntos_muestreo # Use gdf_puntos_muestreo which should have DEM


else:
    print("Error: Cannot proceed with feature extraction. Ensure gdf_puntos_muestreo and roi_geometry are loaded correctly.")
    data_for_modeling = None # Set to None if data loading failed

## Handle missing values

Address any missing data in the combined DataFrame.


In [ ]:
# Inspect missing values
print("Missing values before handling:")
print(data_for_modeling.isnull().sum())

# Identify columns with missing values
missing_columns = data_for_modeling.columns[data_for_modeling.isnull().any()].tolist()
print(f"\nColumns with missing values: {missing_columns}")

# Imputation strategy: Use the mean for numerical columns.
# The missing values are likely in the multi-year NDVI and NDWI columns
# due to cloud cover or lack of satellite data for specific years/seasons/locations.
# Imputing with the mean of available data for that specific index (e.g., mean NDVI for 2022_dry)
# is a reasonable starting point.

# Apply imputation
for col in missing_columns:
    # Check if the column is numerical before attempting mean imputation
    if pd.api.types.is_numeric_dtype(data_for_modeling[col]):
        mean_value = data_for_modeling[col].mean()
        data_for_modeling[col].fillna(mean_value, inplace=True)
        print(f"Imputed missing values in '{col}' with the mean ({mean_value:.4f}).")
    else:
        print(f"Column '{col}' is not numerical. Skipping mean imputation.")


# Verify no missing values remain
print("\nMissing values after handling:")
print(data_for_modeling.isnull().sum())

# Display the first few rows to see the result
display(data_for_modeling.head())

## Define features and target

Separate the features (predictor variables) from the target variable (presence/absence of groundwater). Separate the target variable 'presencia' into y and the remaining feature columns into X. Include 'DEM' and 'TWI', and the multi-year NDVI and NDWI columns as features.

In [ ]:
# Define the target variable
y = data_for_modeling['presencia']

# Define the features (predictor variables)
# Exclude 'Site', 'Longitud', 'Latitud', 'PresFrea', 'presencia', and 'geometry'
# Include 'DEM', 'TWI', and all the multi-year NDVI and NDWI columns, and 'block_id'
feature_columns = [col for col in data_for_modeling.columns if col not in ['Site', 'Longitud', 'Latitud', 'PresFrea', 'presencia', 'geometry']]

X = data_for_modeling[feature_columns]

print("Features (X) shape:", X.shape)
print("Target (y) shape:", y.shape)

# Display the first few rows of the features and target
print("\nFeatures (X) head:")
display(X.head())

print("\nTarget (y) head:")
display(y.head())

## Define Spatial Blocks using K-Means Clustering

Apply K-Means clustering to the point coordinates to create spatial blocks.

In [ ]:
import numpy as np
import pandas as pd

# Define the number of blocks you want to create.
# A smaller number of blocks might increase the chance of having both classes in each block,
# but reduces the number of cross-validation folds.
n_blocks = 5 # You can adjust this number

print(f"Defining spatial blocks based on latitude quantiles with {n_blocks} blocks.")

# Create blocks based on latitude quantiles.
# This divides the data points into blocks based on their latitude,
# aiming for a roughly equal number of points in each block.
data_for_modeling['block_id'] = pd.qcut(data_for_modeling['Latitud'], q=n_blocks, labels=False) + 1

print(f"Spatial blocks created based on {n_blocks} latitude quantiles.")
print("Added 'block_id' column to data_for_modeling.")

# Check the distribution of data points per block
print("\nDistribution of data points per block:")
print(data_for_modeling['block_id'].value_counts().sort_index())

# Check the distribution of classes within each block
print("\nDistribution of classes within each block:")
print(data_for_modeling.groupby('block_id')['presencia'].value_counts().unstack(fill_value=0))

# Display the first few rows with the new 'block_id'
display(data_for_modeling.head())

## Split data

Split the data into training and testing sets.

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets (e.g., 80% train, 20% test)
# Using a fixed random_state for reproducibility
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("Training features shape:", X_train.shape)
print("Testing features shape:", X_test.shape)
print("Training target shape:", y_train.shape)
print("Testing target shape:", y_test.shape)

# Check the distribution of the target variable in train and test sets
print("\nTraining target distribution:")
print(y_train.value_counts(normalize=True))

print("\nTesting target distribution:")
print(y_test.value_counts(normalize=True))

## Feature preparation for modeling

We will apply a Random Forest model with SMOTE for balancing and Block Cross-Validation with hyperparameter tuning using GridSearch on the combined DataFrame, after unifying the 'DEM_x' and 'DEM_y' columns into a single 'DEM' column, and the 'TWI_x' and 'TWI_y' columns into a single 'TWI' column.

## Define features and target

Separate the features (predictor variables) from the target variable (presence/absence of groundwater). Separate the target variable 'presencia' into y and the remaining feature columns into X.


## Address class imbalance with SMOTE

Apply SMOTE to the training data to handle class imbalance.

In [ ]:
from imblearn.over_sampling import SMOTE

# Apply SMOTE to the training data only
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

print("Shape of X_train before SMOTE:", X_train.shape)
print("Shape of X_train after SMOTE:", X_train_res.shape)
print("Shape of y_train before SMOTE:", y_train.shape)
print("Shape of y_train after SMOTE:", y_train_res.shape)

print("\nDistribution of target variable before SMOTE:")
print(y_train.value_counts())

print("\nDistribution of target variable after SMOTE:")
print(y_train_res.value_counts())

## Define the Random Forest model

Initialize the Random Forest Classifier.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Initialize the Random Forest Classifier
# We'll use default parameters for now, hyperparameter tuning will be done later with GridSearch
rf_model = RandomForestClassifier(random_state=42)

print("Random Forest model initialized.")

## Set up Hyperparameter Tuning with GridSearch and Block Cross-Validation

Define the parameter grid for GridSearch and configure it with Block Cross-Validation.

In [ ]:
from sklearn.model_selection import GridSearchCV, KFold, GroupKFold
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE # Import SMOTE
from imblearn.pipeline import Pipeline # Import Pipeline
import pandas as pd # Import pandas to access the DataFrame

# Define the parameter grid to tune for the RandomForestClassifier
# Note: Parameters for SMOTE can also be tuned if needed, by adding steps to the pipeline.
param_grid = {
    'classifier__n_estimators': [100, 200, 300],
    'classifier__max_depth': [None, 10, 20, 30],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 4],
    'classifier__criterion': ['gini', 'entropy']
    # Example of tuning SMOTE parameters:
    # 'smote__k_neighbors': [3, 5, 7]
}

# Create a pipeline that first applies SMOTE and then trains the RandomForestClassifier
# The SMOTE step will only be applied to the training data within each CV fold.
pipeline = Pipeline([
    ('smote', SMOTE(random_state=42)), # Use SMOTE with a fixed random_state
    ('classifier', RandomForestClassifier(random_state=42)) # Use RandomForestClassifier
])

# For Block Cross-Validation, we use GroupKFold with the 'block_id' column.
# We need to provide the groups (block_id) to the split method of GroupKFold.

# Ensure 'block_id' is in the training data (X_train).
# Assuming 'block_id' was added to data_for_modeling before the train-test split
# and is therefore included in X_train.
if 'block_id' in X_train.columns:
    # Get the block IDs for the original training data (X_train)
    groups_for_cv = X_train['block_id']
    # Drop 'block_id' from X_train as it's used for grouping, not as a feature for the model
    X_train_no_block = X_train.drop(columns=['block_id'])

    # Set up GroupKFold
    # n_splits should be less than or equal to the number of unique blocks
    n_splits_cv = min(5, groups_for_cv.nunique()) # Use up to 5 splits, not more than unique blocks
    cv_strategy = GroupKFold(n_splits=n_splits_cv)
    print(f"GroupKFold Cross-Validation set up with {n_splits_cv} splits based on 'block_id'.")

    # Set up GridSearchCV
    # We use the pipeline as the estimator and the original training data (X_train_no_block, y_train)
    # The 'groups' parameter will be provided to the fit method.
    grid_search = GridSearchCV(estimator=pipeline,
                               param_grid=param_grid,
                               cv=cv_strategy,
                               scoring='f1',  # Using F1-score as the scoring metric
                               n_jobs=-1,     # Use all available cores
                               verbose=2)    # Increase verbosity to see progress

    print("GridSearch with Pipeline (SMOTE + RandomForest) and GroupKFold Cross-Validation set up.")
    print("SMOTE will be applied within each CV fold's training subset.")

else:
    print("Error: 'block_id' not found in X_train. Cannot proceed with GroupKFold.")
    grid_search = None # Cannot set up grid search without blocks

## Train the model with GridSearch

Train the Random Forest model using the training data with the configured GridSearch to find the best hyperparameters.

In [ ]:
# Train the model using GridSearchCV on the original training data
print("Starting GridSearch training...")
# Provide the original training data (X_train_no_block, y_train) and the groups_for_cv
# X_train_no_block is X_train with the 'block_id' column dropped.
grid_search.fit(X_train_no_block, y_train, groups=groups_for_cv)


print("\nGridSearch training complete.")
print("Best parameters found:", grid_search.best_params_)
print("Best F1-score on training data:", grid_search.best_score_)

# Store the best model from the pipeline
best_pipeline_model = grid_search.best_estimator_
print("\nBest Pipeline model (SMOTE + RandomForest) stored.")

# You can access the best RandomForest model from the pipeline like this:
best_rf_model = best_pipeline_model.named_steps['classifier']
print("Best RandomForest model extracted from the pipeline.")

## Evaluate the model

Evaluate the trained model using the test set with appropriate metrics (e.g., accuracy, precision, recall, F1-score, ROC AUC).

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

# Make predictions on the test set using the best pipeline model
# We need to drop the 'block_id' column from X_test before making predictions,
# as the model was trained without it.
if 'block_id' in X_test.columns:
    X_test_no_block = X_test.drop(columns=['block_id'])
else:
    X_test_no_block = X_test # Use X_test directly if block_id is not present


y_pred = best_pipeline_model.predict(X_test_no_block)
y_prob = best_pipeline_model.predict_proba(X_test_no_block)[:, 1] # Get probabilities for the positive class

# Evaluate the model using various metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_prob)
conf_matrix = confusion_matrix(y_test, y_pred)


print("Model Evaluation on the Test Set:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"ROC AUC: {roc_auc:.4f}")
print("\nConfusion Matrix:")
print(conf_matrix)


# Interpret the confusion matrix:
# [[True Negatives  False Positives]
#  [False Negatives True Positives]]
print("\nConfusion Matrix Interpretation:")
print(f"True Positives (correctly predicted presence): {conf_matrix[1, 1]}")
print(f"True Negatives (correctly predicted absence): {conf_matrix[0, 0]}")
print(f"False Positives (predicted presence, but actually absence): {conf_matrix[0, 1]}")
print(f"False Negatives (predicted absence, but actually presence): {conf_matrix[1, 0]}")

# Apply RF trained to original ROI
Predict the presence of groundwater using a trained Random Forest model with SMOTE and Block Cross-Validation on the original area of interest defined by the Earth Engine asset 'users/facu_ruarte/Sur_SanJuan_export'.

## Define prediction area geometry

Load the geometry for the original prediction area from the Earth Engine asset.


In [ ]:
# Define the Earth Engine asset ID for the original prediction area
original_area_asset_id = 'users/facu_ruarte/Sur_SanJuan_export'

# Load the asset as an Earth Engine FeatureCollection
try:
    original_area_feature_collection = ee.FeatureCollection(original_area_asset_id)
    print(f"Successfully loaded original area asset: {original_area_asset_id}")

    # Extract the geometry from the FeatureCollection.
    # Using .geometry().bounds() to get the bounding box geometry
    prediction_area_geometry = original_area_feature_collection.geometry() # Using .geometry() to get the merged geometry
    print("Extracted geometry for the original prediction area.")

except ee.EEException as e:
    print(f"Error loading Earth Engine asset: {e}")
    print(f"Please ensure the asset ID '{original_area_asset_id}' is correct and the asset is publicly accessible or in your Earth Engine account.")
    prediction_area_geometry = None # Set geometry to None if loading fails
except Exception as e:
    print(f"An unexpected error occurred: {e}")
    prediction_area_geometry = None # Set geometry to None if any other error occurs

# Check if the geometry was successfully loaded
if prediction_area_geometry is not None:
    print("Prediction area geometry is loaded and ready for use.")
else:
    print("Failed to load prediction area geometry.")

## Extract features for prediction area

### Subtask:
Obtain the necessary predictor features (DEM, TWI, multi-year NDVI, NDWI) as raster layers for the entire prediction area by calculating or loading them from Earth Engine assets and stacking them into a single multi-band image.


## Extract features for prediction area

Obtain the necessary predictor features (DEM, TWI, multi-year NDVI, NDWI) as raster layers for the entire prediction area.


In [ ]:
# Define the Earth Engine asset ID for the original prediction area
original_area_asset_id = 'users/facu_ruarte/Sur_SanJuan_Export'

# Load the asset as an Earth Engine FeatureCollection
try:
    original_area_feature_collection = ee.FeatureCollection(original_area_asset_id)
    print(f"Successfully loaded original area asset: {original_area_asset_id}")

    # Extract the geometry from the FeatureCollection.
    # Using .geometry() to get the merged geometry of all features in the collection
    prediction_area_geometry = original_area_feature_collection.geometry()
    print("Extracted geometry for the original prediction area.")

except ee.EEException as e:
    print(f"Error loading Earth Engine asset: {e}")
    print(f"Please ensure the asset ID '{original_area_asset_id}' is correct and the asset is publicly accessible or in your Earth Engine account.")
    prediction_area_geometry = None # Set geometry to None if loading fails
except Exception as e:
    print(f"An unexpected error occurred: {e}")
    prediction_area_geometry = None # Set geometry to None if any other error occurs

# Check if the geometry was successfully loaded
if prediction_area_geometry is not None:
    print("Prediction area geometry is loaded and ready for use.")
else:
    print("Failed to load prediction area geometry.")


Now we can proceed with obtaining the necessary predictor features (DEM, TWI, multi-year NDVI, NDWI) as raster layers for this prediction area.



In [ ]:
import math # Ensure math is imported for calculations
import ee # Ensure ee is imported

# --- 1. Load and Filter DEM (SRTM) ---
srtm_image_id = 'USGS/SRTMGL1_003'
srtm_image = ee.Image(srtm_image_id)
if prediction_area_geometry is not None:
    # Select elevation and cast to Float32
    dem_image = srtm_image.clip(prediction_area_geometry).select('elevation').toFloat()
    print("DEM image loaded, clipped, and cast to Float32.")
else:
    print("Warning: prediction_area_geometry is not available. Cannot clip DEM image. Casting to Float32.")
    dem_image = srtm_image.select('elevation').toFloat() # Use full image as fallback, cast to Float32


# --- 2. Calculate Slope ---
if dem_image is not None:
    slope_image = ee.Terrain.slope(dem_image) # Slope calculation also works on Float
    print("Slope image calculated.")
else:
    print("Warning: DEM image not available. Cannot calculate slope.")
    slope_image = None # Set slope to None if DEM is not available

# --- 3. Load and Clip Flow Accumulation ---
flow_accumulation_asset_id = 'projects/identificacion-de-bosques/assets/acumulacion_flujo_expanded'
try:
    flow_accumulation_ee_image = ee.Image(flow_accumulation_asset_id)
    if prediction_area_geometry is not None:
        # Select band and cast to Float32
        clipped_flow_accumulation = flow_accumulation_ee_image.clip(prediction_area_geometry).select('b1').toFloat()
        print("Flow accumulation image loaded, clipped, and cast to Float32.")
    else:
        print("Warning: prediction_area_geometry is not available. Cannot clip flow accumulation image. Casting to Float32.")
        clipped_flow_accumulation = flow_accumulation_ee_image.select('b1').toFloat() # Use full image as fallback, cast to Float32

except ee.EEException as e:
    print(f"Error loading flow accumulation asset: {e}")
    clipped_flow_accumulation = None # Set to None if loading fails
except Exception as e:
    print(f"An unexpected error occurred while loading flow accumulation: {e}")
    clipped_flow_accumulation = None


# --- 4. Calculate TWI ---
twi_image = None # Initialize TWI image as None
if clipped_flow_accumulation is not None and slope_image is not None:
    # Ensure slope is in radians and avoid division by zero
    slope_radians = slope_image.divide(180).multiply(ee.Number(math.pi))
    # Replace zero or near-zero slopes with a small positive number
    min_slope_deg = 0.001
    slope_corrected = slope_image.max(min_slope_deg)
    slope_radians_corrected = slope_corrected.divide(180).multiply(ee.Number(math.pi))
    tan_slope = slope_radians_corrected.tan()

    # Replace zero or near-zero flow accumulation values
    min_flow_acc = 1e-5 # Use a very small positive number
    flow_accumulation_corrected = clipped_flow_accumulation.max(min_flow_acc) # Should already be Float32

    # Calculate TWI: ln(Flow Accumulation / tan(Slope))
    twi_image = flow_accumulation_corrected.divide(tan_slope).log().rename('TWI').toFloat() # Ensure TWI is Float32
    print("TWI image calculated and cast to Float32.")
else:
    print("Warning: Cannot calculate TWI. Flow accumulation or slope image is missing.")


# --- 5. Generate Multi-year Median NDVI and NDWI Collections ---
# Function to calculate NDVI and NDWI (assuming add_indices is defined earlier)
# def add_indices(image): ... # This function should also cast bands to Float if needed,
# but normalizedDifference usually results in Float.

def get_season_median_predict(year, months, season_name, geometry):
    start = ee.Date.fromYMD(year, months[0], 1)
    end = ee.Date.fromYMD(year, months[-1], 30)
    collection = (ee.ImageCollection("COPERNICUS/S2_SR")
                  .filterDate(start, end)
                  .filterBounds(geometry)
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))
                  .map(add_indices) # Assuming add_indices is defined and outputs Float bands
                  .select(['NDVI', 'NDWI']))
    median = collection.median()
    if geometry is not None:
        median = median.clip(geometry)
    return median.set({'year': year, 'season': season_name}).toFloat() # Ensure median image is Float32


dry_months = [2, 3, 4]
wet_months = [7, 8, 9]
prediction_years = range(2019, 2026) # Use the same years as in training data

dry_median_images = []
wet_median_images = []

if prediction_area_geometry is not None:
    for year in prediction_years:
        dry_median = get_season_median_predict(year, dry_months, 'dry', prediction_area_geometry)
        wet_median = get_season_median_predict(year, wet_months, 'wet', prediction_area_geometry)
        dry_median_images.append(dry_median)
        wet_median_images.append(wet_median)

    print("Multi-year median NDVI and NDWI images generated and cast to Float32.")
else:
    print("Warning: prediction_area_geometry not available. Cannot generate multi-year median images.")


# --- 6. Stack Features ---
# Create a list of images to stack
images_to_stack = []

if dem_image is not None:
    images_to_stack.append(dem_image) # Should already be Float32
    print("Added DEM to stack list.")
if twi_image is not None:
    images_to_stack.append(twi_image) # Should already be Float32
    print("Added TWI to stack list.")

# Add multi-year NDVI and NDWI images to the list
for img in dry_median_images:
    # Select NDVI and NDWI bands and rename. Ensure they are Float32.
    # get_season_median_predict should return Float32 images.
    try:
        year = img.get('year').getInfo()
        bands = img.bandNames().getInfo()
        if 'NDVI' in bands and 'NDWI' in bands:
            dry_ndvi = img.select('NDVI').rename(f'NDVI_{year}_dry')
            dry_ndwi = img.select('NDWI').rename(f'NDWI_{year}_dry')
            images_to_stack.extend([dry_ndvi, dry_ndwi])
            print(f"Added NDVI_{year}_dry and NDWI_{year}_dry (Float32) to stack list.")
        else:
             print(f"Warning: NDVI or NDWI band not found in dry image for year {year}. Bands found: {bands}. Skipping.")
    except Exception as e:
        print(f"Error processing dry median image: {e}. Skipping.")


for img in wet_median_images:
    # Select NDVI and NDWI bands and rename. Ensure they are Float32.
    # get_season_median_predict should return Float32 images.
     try:
        year = img.get('year').getInfo()
        bands = img.bandNames().getInfo()
        if 'NDVI' in bands and 'NDWI' in bands:
            wet_ndvi = img.select('NDVI').rename(f'NDVI_{year}_wet')
            wet_ndwi = img.select('NDWI').rename(f'NDWI_{year}_wet')
            images_to_stack.extend([wet_ndvi, wet_ndwi])
            print(f"Added NDVI_{year}_wet and NDWI_{year}_wet (Float32) to stack list.")
        else:
            print(f"Warning: NDVI or NDWI band not found in wet image for year {year}. Bands found: {bands}. Skipping.")
     except Exception as e:
        print(f"Error processing wet median image: {e}. Skipping.")


# Check if there are images to stack
if images_to_stack:
    # Stack the images
    stacked_features_image = ee.Image(images_to_stack)
    print("\nAll features stacked into a single image.")

    # Get the current band names of the stacked image
    current_stacked_band_names = stacked_features_image.bandNames().getInfo()
    print("\nCurrent stacked image band names:", current_stacked_band_names)

    # Create the list of new band names, ensuring it matches the number of bands
    # Based on the previous output, it seems the stack might be missing one band.
    # Let's regenerate the expected band names based on the successfull additions to the list.
    # Assuming X_train_no_block represents the training features used by the model
    if 'X_train_no_block' in locals() and X_train_no_block is not None:
        expected_training_feature_names = X_train_no_block.columns.tolist()
    elif 'X' in locals() and X is not None and 'block_id' in X.columns:
        expected_training_feature_names = [col for col in X.columns if col != 'block_id']
    elif 'X' in locals() and X is not None:
        expected_training_feature_names = X.columns.tolist()
    else:
        print("Warning: Could not reliably determine training feature names. Proceeding with caution.")
        # Fallback: manually construct expected band names based on the stacking process
        expected_training_feature_names = ['DEM', 'TWI'] + [f'NDVI_{year}_dry' for year in prediction_years] + [f'NDWI_{year}_dry' for year in prediction_years] + [f'NDVI_{year}_wet' for year in prediction_years] + [f'NDWI_{year}_wet' for year in prediction_years]
        # Need to filter this list based on which images were actually added to images_to_stack
        # This is getting complex. Let's just use the current stacked band names and rename 'elevation' if present.
        print("Using current stacked band names for renaming logic.")


    # Create a list of new band names based on the current stacked band names
    new_band_names = []
    for band_name in current_stacked_band_names:
         if band_name == 'elevation':
             new_band_names.append('DEM')
         else:
             new_band_names.append(band_name) # Keep other names as they are assumed to be correct


    print("\nGenerated new band names list for renaming:", new_band_names)
    print("Number of new band names:", len(new_band_names))
    print("Number of bands in stacked image:", len(current_stacked_band_names))


    # Rename the bands only if the number of names matches the number of bands
    if len(new_band_names) == len(current_stacked_band_names):
        try:
            stacked_features_image = stacked_features_image.rename(new_band_names)
            print("\nSuccessfully renamed bands in the stacked image.")

            # Final verification of band names
            stacked_band_names_final = stacked_features_image.bandNames().getInfo()
            print("\nFinal stacked image band names:", stacked_band_names_final)
            # Re-evaluate expected_training_feature_names if needed for comparison
            if 'X_train_no_block' in locals() and X_train_no_block is not None:
                 expected_training_feature_names_final = X_train_no_block.columns.tolist()
            else:
                 # Fallback or use the previous expected list if available
                 expected_training_feature_names_final = expected_training_feature_names # Use the list generated earlier if possible

            print("Training feature names (excluding block_id):", expected_training_feature_names_final)

            missing_bands_in_stack = [name for name in expected_training_feature_names_final if name not in stacked_band_names_final]
            extra_bands_in_stack = [name for name in stacked_band_names_final if name not in expected_training_feature_names_final]

            if not missing_bands_in_stack:
                print("\nAll training features have corresponding bands in the stacked image after final rename.")
            else:
                print(f"\nWarning: The following training features are missing in the stacked image after final rename: {missing_bands_in_stack}")

            if not extra_bands_in_stack:
                 print("No extra bands in the stacked image compared to training features after final rename.")
            else:
                 print(f"Warning: The following extra bands are present in the stacked image after final rename: {extra_bands_in_stack}")


        except ee.EEException as e:
            print(f"Error during final renaming: {e}")
        except Exception as e:
            print(f"An unexpected error occurred during final renaming: {e}")

    else:
        print("\nError: Number of new band names does not match the number of bands in the stacked image. Renaming skipped.")


else:
    print("\nError: No images available to stack. Cannot create stacked features image.")
    stacked_features_image = None

In [ ]:
import math # Ensure math is imported for calculations
import ee # Ensure ee is imported

# Print the band names of the individual images before stacking to diagnose the issue
print("Band names of individual images before stacking:")

if dem_image is not None:
    print("DEM image bands:", dem_image.bandNames().getInfo())
else:
    print("DEM image is None.")

if twi_image is not None:
    print("TWI image bands:", twi_image.bandNames().getInfo())
else:
    print("TWI image is None.")

for i, img in enumerate(dry_median_images):
    try:
        print(f"Dry median image {i+1} (Year {img.get('year').getInfo()}):", img.bandNames().getInfo())
    except Exception as e:
        print(f"Error getting bands for dry median image {i+1}: {e}")

for i, img in enumerate(wet_median_images):
    try:
        print(f"Wet median image {i+1} (Year {img.get('year').getInfo()}):", img.bandNames().getInfo())
    except Exception as e:
        print(f"Error getting bands for wet median image {i+1}: {e}")

# Re-stack the images, carefully checking the process
images_to_stack = []

if dem_image is not None:
    images_to_stack.append(dem_image) # Should already be Float32 from previous step
    print("\nAdded DEM to stack list.")
if twi_image is not None:
    images_to_stack.append(twi_image) # Should already be Float32 from previous step
    print("Added TWI to stack list.")

# Add multi-year NDVI and NDWI images to the list, ensuring bands exist and are Float32
for img in dry_median_images:
    try:
        year = img.get('year').getInfo()
        bands = img.bandNames().getInfo()
        if 'NDVI' in bands and 'NDWI' in bands:
            # Select bands and rename. Ensure they are Float32 (should be from get_season_median_predict)
            dry_ndvi = img.select('NDVI').rename(f'NDVI_{year}_dry')
            dry_ndwi = img.select('NDWI').rename(f'NDWI_{year}_dry')
            images_to_stack.extend([dry_ndvi, dry_ndwi])
            print(f"Added NDVI_{year}_dry and NDWI_{year}_dry (Float32) to stack list.")
        else:
             print(f"Warning: NDVI or NDWI band not found in dry image for year {year}. Bands found: {bands}. Skipping.")
    except Exception as e:
        print(f"Error processing dry median image: {e}. Skipping.")


for img in wet_median_images:
    try:
        year = img.get('year').getInfo()
        bands = img.bandNames().getInfo()
        if 'NDVI' in bands and 'NDWI' in bands:
            # Select bands and rename. Ensure they are Float32 (should be from get_season_median_predict)
            wet_ndvi = img.select('NDVI').rename(f'NDVI_{year}_wet')
            wet_ndwi = img.select('NDWI').rename(f'NDWI_{year}_wet')
            images_to_stack.extend([wet_ndvi, wet_ndwi])
            print(f"Added NDVI_{year}_wet and NDWI_{year}_wet (Float32) to stack list.")
        else:
            print(f"Warning: NDVI or NDWI band not found in wet image for year {year}. Bands found: {bands}. Skipping.")
    except Exception as e:
        print(f"Error processing wet median image: {e}. Skipping.")


# Check if there are images to stack
if images_to_stack:
    # Stack the images
    stacked_features_image = ee.Image(images_to_stack)
    print("\nAll features stacked into a single image.")

    # Get the current band names of the stacked image
    current_stacked_band_names = stacked_features_image.bandNames().getInfo()
    print("\nCurrent stacked image band names:", current_stacked_band_names)

    # Create the list of new band names based on the current stacked band names
    new_band_names = []
    for band_name in current_stacked_band_names:
         if band_name == 'elevation':
             new_band_names.append('DEM')
         else:
             new_band_names.append(band_name) # Keep other names as they are assumed to be correct


    print("\nGenerated new band names list for renaming:", new_band_names)
    print("Number of new band names:", len(new_band_names))
    print("Number of bands in stacked image:", len(current_stacked_band_names))


    # Rename the bands only if the number of names matches the number of bands
    if len(new_band_names) == len(current_stacked_band_names):
        try:
            stacked_features_image = stacked_features_image.rename(new_band_names)
            print("\nSuccessfully renamed bands in the stacked image.")

            # Final verification of band names
            stacked_band_names_final = stacked_features_image.bandNames().getInfo()
            print("\nFinal stacked image band names:", stacked_band_names_final)
            # Re-evaluate expected_training_feature_names if needed for comparison
            if 'X_train_no_block' in locals() and X_train_no_block is not None:
                 expected_training_feature_names_final = X_train_no_block.columns.tolist()
            elif 'X' in locals() and X is not None and 'block_id' in X.columns:
                expected_training_feature_names_final = [col for col in X.columns if col != 'block_id']
            elif 'X' in locals() and X is not None:
                 expected_training_feature_names_final = X.columns.tolist()
            else:
                 print("Warning: Could not reliably determine training feature names. Cannot fully verify band names.")
                 expected_training_feature_names_final = [] # Cannot verify without training features


            print("Training feature names (excluding block_id):", expected_training_feature_names_final)

            missing_bands_in_stack = [name for name in expected_training_feature_names_final if name not in stacked_band_names_final]
            extra_bands_in_stack = [name for name in stacked_band_names_final if name not in expected_training_feature_names_final]

            if expected_training_feature_names_final: # Only print if training feature names could be determined
                if not missing_bands_in_stack:
                    print("\nAll training features have corresponding bands in the stacked image after final rename.")
                else:
                    print(f"\nWarning: The following training features are missing in the stacked image after final rename: {missing_bands_in_stack}")

                if not extra_bands_in_stack:
                     print("No extra bands in the stacked image compared to training features after final rename.")
                else:
                     print(f"Warning: The following extra bands are present in the stacked image after final rename: {extra_bands_in_stack}")
            else:
                print("\nSkipping final band name verification as training feature names could not be determined.")


        except ee.EEException as e:
            print(f"Error during final renaming: {e}")
        except Exception as e:
            print(f"An unexpected error occurred during final renaming: {e}")

    else:
        print("\nError: Number of new band names does not match the number of bands in the stacked image. Renaming skipped.")


else:
    print("\nError: No images available to stack. Cannot create stacked features image.")
    stacked_features_image = None

## Prepare feature data for prediction

Convert the feature image into a format suitable for prediction with the trained model (e.g., an array or a flattened representation).


In [ ]:
import numpy as np
import geemap # Ensure geemap is imported
import ee # Ensure ee is imported

# Convert the stacked Earth Engine image to a numpy array
# Use the prediction_area_geometry for the region
# Use a suitable scale (e.g., 30 meters, matching the SRTM resolution)
try:
    # Get the scale from one of the images if available, or use a default
    if stacked_features_image is not None:
        # Attempt to get nominal scale, default to 30 if not available or problematic
        scale = stacked_features_image.select(0).projection().nominalScale().getInfo()
        if scale is None or scale <= 0:
            scale = 30 # Fallback to 30 if nominalScale is not valid
        print(f"Using prediction scale: {scale} meters.")
    else:
        scale = 30 # Default scale if stacked_features_image is None
        print(f"Warning: stacked_features_image is None. Using default scale: {scale} meters.")

    # Ensure prediction_area_geometry is not None before proceeding
    if prediction_area_geometry is not None:
        # Get the GeoJSON representation of the prediction area geometry
        prediction_area_geojson = prediction_area_geometry.getInfo()
        print("Obtained GeoJSON representation of the prediction area geometry.")

        # Pass the stacked image as the EE object and the GeoJSON as the region
        features_array = geemap.ee_to_numpy(
            ee_object=stacked_features_image, # Pass the stacked image here
            region=prediction_area_geojson, # Use the GeoJSON as the region
            scale=scale,
            # Specify the data type to ensure compatibility with the model
            dtype=np.float32 # Using float32 for efficiency and compatibility
        )
        print("\nEarth Engine image converted to numpy array.")
        print("Original numpy array shape:", features_array.shape)

        # Reshape the numpy array for prediction
        # The current shape is (rows, columns, features)
        # We need to reshape it to (number of pixels, number of features)
        # Number of pixels = rows * columns
        # Number of features = the last dimension of the array
        if features_array is not None and features_array.ndim == 3:
            rows, cols, n_features = features_array.shape
            features_array_reshaped = features_array.reshape(rows * cols, n_features)
            print("Numpy array reshaped for prediction.")
            print("Reshaped numpy array shape:", features_array_reshaped.shape)
        elif features_array is not None and features_array.ndim == 2:
             # Handle case where array might be 2D (e.g., single band image)
             rows, cols = features_array.shape
             n_features = 1
             features_array_reshaped = features_array.reshape(rows * cols, n_features)
             print("Numpy array reshaped (2D case) for prediction.")
             print("Reshaped numpy array shape:", features_array_reshaped.shape)
        else:
            print("Error: Numpy array does not have expected dimensions (3 or 2). Cannot reshape.")
            features_array_reshaped = None # Set to None if reshaping fails


        # Handle potential nodata values (often represented by NaNs)
        # Replace NaN values with a default value, e.g., the mean of the valid data
        if features_array_reshaped is not None:
             if np.isnan(features_array_reshaped).any():
                 print("\nHandling NaN values in the reshaped array...")
                 # Compute mean only on non-NaN values
                 mean_per_feature = np.nanmean(features_array_reshaped, axis=0)
                 # Replace NaNs with the calculated mean for each feature column
                 # Use np.nan_to_num for a robust replacement
                 features_array_reshaped = np.nan_to_num(features_array_reshaped, nan=mean_per_feature)
                 print("NaN values replaced with column means.")
             else:
                 print("\nNo NaN values found in the reshaped array.")

        # Verify the data type
        if features_array_reshaped is not None:
            print("Data type of reshaped array:", features_array_reshaped.dtype)
            if features_array_reshaped.dtype != np.float32:
                 print("Warning: Reshaped array data type is not float32.") # Optional warning


    else:
        print("Error: prediction_area_geometry is None. Cannot convert Earth Engine image to numpy array.")
        features_array = None
        features_array_reshaped = None


except Exception as e:
    print(f"An error occurred during Earth Engine to numpy conversion or reshaping: {e}")
    features_array = None
    features_array_reshaped = None

# At this point, features_array_reshaped contains the pixel data in the correct shape for prediction
# It's a numpy array of shape (number of pixels, number of features)
# Make sure features_array_reshaped is not None before proceeding to the prediction step.

## Export stacked features image

Export the `stacked_features_image` from Earth Engine to a GeoTIFF file in your Google Drive.


In [ ]:
# Define the parameters for the export task
task_description = 'Groundwater_Prediction_Features_Export'
export_folder = 'earth_engine_exports'
export_file_name = 'predicted_features'
export_scale = 30  # Use a scale that matches your analysis (e.g., 30 meters for SRTM)

# Ensure stacked_features_image is not None and prediction_area_geometry is not None
if stacked_features_image is not None and prediction_area_geometry is not None:
    # Create the export task
    export_task = ee.batch.Export.image.toDrive(
        image=stacked_features_image,
        description=task_description,
        folder=export_folder,
        fileNamePrefix=export_file_name,
        scale=export_scale,
        region=prediction_area_geometry, # Pass the Earth Engine Geometry object directly
        fileFormat='GeoTIFF',
        crs='EPSG:4326' # Specify a coordinate reference system
    )

    # Start the export task
    export_task.start()

    print(f"Earth Engine export task '{task_description}' started.")
    print(f"Check the Earth Engine Tasks tab to monitor progress.")

else:
    print("Error: stacked_features_image or prediction_area_geometry is not available. Cannot start export task.")

## Make Predictions
Use the trained model (`best_pipeline_model`) to predict the presence/absence or probability for each pixel in the prediction area.

In [ ]:
# Ensure the trained model and the reshaped features array are available
if 'best_pipeline_model' in locals() and best_pipeline_model is not None and \
   'features_array_reshaped' in locals() and features_array_reshaped is not None:

    print("Making predictions using the trained model...")

    # Make predictions (either classes or probabilities)
    # To get predicted classes (0 or 1):
    # predictions = best_pipeline_model.predict(features_array_reshaped)

    # To get predicted probabilities (useful for visualization or further analysis):
    # The output is an array of shape (n_samples, n_classes).
    # We want the probability of the positive class (class 1).
    prediction_probabilities = best_pipeline_model.predict_proba(features_array_reshaped)[:, 1]

    print("Predictions made successfully.")
    print("Prediction probabilities shape:", prediction_probabilities.shape)

    # Store the prediction results (probabilities in this case)
    prediction_results = prediction_probabilities

else:
    print("Error: Trained model (best_pipeline_model) or reshaped features (features_array_reshaped) not available. Cannot make predictions.")
    prediction_results = None # Set to None if prediction fails